# 5-1. 이동, 확대/축소, 회전

 - 기하학적 변환(geometric transform): 기존의 영상을 원하는 모양이나 방향 등으로 변환하기 위해 각 픽셀을 새로운 위치로 옮기는 작업
 - 행렬은 새로운 위치의 좌표를 구하기에 가장 좋은 방법.

#### 5-1-1. 이동
- 2차원 공간에서의 물체 이동: 원래 있던 좌표에 이동시키려는 거리만큼 더해서 이동할 좌표를 구함.
- 한 점 p(x, y)에서 새 좌표 p(x', y')로 옮기는 것을 수식으로 표현

                                x' = x + dx
                                y' = y + dy

- 위 식은 곧 다음 행렬식과 같음( (2*3) * (3*1) 행렬)
                        
                            x' = 1 0 dx    x
                                           y
                            y' = 0 1 dy    1

- 행렬식 중에서도 x, y는 이미 원본 이미지의 좌표 값으로 제공되므로, 2*3 변환행렬만 전달하면 연산 가능.
- OpenCV의 영상 좌표를 2*3행렬로 변환하는 함수.

      - dst = cv2.warpAffine(src, mtrx, dsize [, dst, flags, borderMode, borderValue])
          * src: 원본 영상. numpy 배열
          * mtrx: 2*3 변환행렬. Numpy 배열, dtype = float32
          * dsize: 결과 이미지 크기. tuple(width, height)
          * flags: 보간법 알고리즘 선택 플래그
            * cv2.INTER_LINEAR: 기본값. 인접한 4개 픽셀값에 거리 가중치 사용.
            * cv2.INTER_NEAREST: 가장 가까운 픽셀값 사용.
            * cv2.INTER_AREA: 픽셀 영역 관계를 이용한 재샘플링
            * cv2.INTER_CUBIC: 인접한 16개 픽셀 값에 거리 가중치 사용
            * cv2.INTER_LANCZ0S4: 인접한 8개 픽셀을 이용한 란초의 알고리즘.
          * borderMode: 외곽 영역 보정 플래그
            * cv2.BORDER_CONSTANT: 고정 색상값(999|12345|999)
            * cv2.BORDER_REPLICATE: 가장자리 복제(111|12345|555)
            * cv2.BORDER_WRAP: 반복(345|12345|123)
            * cv2.BORDER_REFLECT: 반사(321|12345|543)
          * borderValue: cv2.BORDER_CONSTANT의 경우 사용할 색상값(기본값 = 0)
          * dst: 결과 이미지. Numpy 배열.
          
- src 영상을 mtrx행렬에 따라 변환해서 dsize크기로 만들어 반환함.
- 픽셀 탈락 현상을 보정해주는 보간법 알고리즘과 경계 부분의 보정 방법도 선택할 수 있음.

In [ ]:
#practice. 평행이동.
import cv2
import numpy as np

img = cv2.imread('../img/fish.jpg')
rows, cols = img.shape[0:2]   #영상의 크기

dx, dy = 100, 50    #이동할 픽셀 거리: x로 100픽셀, y로 50픽셀

#변환행렬 생성 -- ①
mtrx = np.float32([[1, 0, dx],
                   [0, 1, dy]])

#단순이동 -- ②
dst = cv2.warpAffine(img, mtrx, (cols + dx, rows + dy))

#탈락된 외곽 픽셀을 파란색으로 보정 -- ③
dst2 = cv2.warpAffine(img, mtrx, (cols + dx, rows + dy), None, \
                      cv2.INTER_LINEAR, cv2.BORDER_CONSTANT, (255, 0, 0))

#탈락된 외곽 픽셀을 원본을 반사시켜셔 보정 -- ④
dst3 = cv2.warpAffine(img, mtrx, (cols + dx, rows + dy), None, \
                     cv2.INTER_LINEAR, cv2.BORDER_REFLECT)

cv2.imshow('original', img)
cv2.imshow('trans', dst)
cv2.imshow('BORDER_CONSTANT', dst2)
cv2.imshow('BORDER_REFLECT', dst3)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 5-1-2. 확대/축소
 - 원래 있던 좌표에 원하는 비율만큼 곱해서 새로운 좌표를 생성.
 - 비율을 가로 방향 a, 세로 방향 b라고 한다면 변환행렬은 아래와 같음.

               [x'] = [a  0  0][x]
                               [y]
               [y'] = [0  b  0][1]
               
 - 굳이 2*3 행렬로 표현한 이유: cv2.warpAffine() 함수와 이동 변환 때문.
 - 행렬 마지막 열에 dx, dy에 해당하는 값을 지정하면 이동도 가능함.
               

In [ ]:
#practice. 행렬을 이용한 확대와 축소
import cv2
import numpy as np

img = cv2.imread('../img/fish.jpg')
height, width = img.shape[:2]

# -- ① 0.5배 축소 변환행렬
m_small = np.float32([[0.5, 0, 0],
                      [0, 0.5, 0]])

# -- ② 2배 확대 변환행렬
m_big = np.float32([[2, 0, 0],
                    [0, 2, 0]])

# -- ③ 보간법 적용 없이 확대/축소
dst1 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)))
dst2 = cv2.warpAffine(img, m_big, (int(height*2), int(width*2)))

# -- ④ 보간법 적용한 확대/축소
dst3 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)), \
                     None, cv2.INTER_AREA) #축소: cv2.INTER_AREA
dst4 = cv2.warpAffine(img, m_big, (int(height * 2), int(width*2)), \
                     None, cv2.INTER_CUBIC) #확대: cv2.INTER_CUBIC, cv2.INTER_LINEAR

# 결과 출력
cv2.imshow("original", img)
cv2.imshow("small", dst1)
cv2.imshow("big", dst2)
cv2.imshow("small INTER_AREA", dst3)
cv2.imshow("big INTER_CUBIC", dst4)
cv2.waitKey(0)
cv2.destroyAllWindows()

 - OpenCV에서 제공하는 확대/축소기능 함수
    
       dst = cv2.resize(src, dsize, dst, fx, fy, interpolation)
         * src: 입력영상. numpy 배열.
         * dsize: 출력 영상 크기(확대 or 축소 목표 크기). 생략하면 fx, fy를 적용
         * (width, height)
         * fx, fy: 크기 배율. 생략하면 dsize를 적용.
         * interpolation: 보간법 알고리즘 선택 플래그(cv2.warpAffine()과 동일)
         * dst: 결과 영상. Numpy 배열.
         
 - dsize로 변경하고 싶은 픽셀 크기를 직접 지정하거나, fx와 fy로 변경할 배율을 지정할 수 있음.
 - dsize와 fx, fy값을 모두 전달하면 dsize만 적용함.

In [ ]:
#practice. cv2.resize()로 확대와 축소.
import cv2
import numpy as np

img = cv2.imread('../img/fish.jpg')
height, width = img.shape[:2]

#크기 지정으로 축소 -- ①
#dst1 = cv2.resize(img, (int(width * 0.5), int(height * 0.5)), None, 0, 0, interpolation = cv2.INTER_AREA)
dst1 = cv2.resize(img, (int(width * 0.5), int(height * 0.5)), interpolation = cv2.INTER_AREA)

#배율 지정으로 확대 -- ②
dst2 = cv2.resize(img, None, None, 2, 2, cv2.INTER_CUBIC)

#결과 출력
cv2.imshow("original", img)
cv2.imshow("small", dst1)
cv2.imshow("big", dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 5-1-3. 회전.

In [ ]:
#practice. 변환행렬로 회전.
import cv2
import numpy as np

img = cv2.imread('../img/fish.jpg')
rows, cols = img.shape[0:2]

#라디안 각도 계산(60진법에서 라디안(radian, 호도법)으로 변경) -- ① 
d45 = 45.0 * np.pi / 180 #45도
d90 = 90.0 * np.pi / 180 #90도

#회전을 위한 변환행렬 생성(numpy의 삼각함수 사용) -- ②
    #각 행렬의 마지막 열에 0 대신 rows //2, -1 * cols // 4, rows를 사용한 이유: 
    #영상의 회전 기준 축이 좌측 상단이 되므로, 회전한 영상은 보여지는 영역 바깥으로 벗어나게 돼서 
    #좌표를 가운데로 옮기기 위한 것으로 회전 축을 지정.
    #변환행렬의 마지막 열은 이동에 사용한다.
m45 = np.float32( [[np.cos(d45), -1 * np.sin(d45), rows // 2],
                   [np.sin(d45), np.cos(d45),      -1 * cols // 4]])
m90 = np.float32( [[np.cos(d90), -1 * np.sin(d90), rows],
                   [np.sin(d90), np.cos(d90),      0]])

#회전 변환행렬 적용
r45 = cv2.warpAffine(img, m45, (cols, rows))
r90 = cv2.warpAffine(img, m90, (rows, cols))

#결과 출력
cv2.imshow("origin", img)
cv2.imshow("45", r45)
cv2.imshow("90", r90)
cv2.waitKey(0)
cv2.destroyAllWindows()

 - 회전을 위한 변환행렬 생성 함수

       - mtrx = cv2.getRotationMatrix2D(center, angle, scale)
          * center: 회전 축 중심 좌표. 튜플(x, y)
          * angle: 회전 각도. 60진법.
          * scale: 확대/축소 배율.

In [ ]:
#practice. 5-5. 회전 변환행렬 구하기
import cv2

img = cv2.imread('../img/fish.jpg')
rows, cols = img.shape[0:2]

#회전을 위한 변환행렬 구하기
#회전 축: 중앙, 각도: 45, 배율: 0.5
m45 = cv2.getRotationMatrix2D((cols/2, rows/2), 45, 0.5)
#회전 축: 중앙, 각도: 90, 배율: 1.5
m90 = cv2.getRotationMatrix2D((cols/2, rows/2), 90, 1.5)

#변환행렬 적용
img45 = cv2.warpAffine(img, m45, (cols, rows))
img90 = cv2.warpAffine(img, m90, (cols, rows))

#결과 출력
cv2.imshow("origin", img)
cv2.imshow("45", img45)
cv2.imshow("90", img90)
cv2.waitKey(0)
cv2.destroyAllWindows()